In [1]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect()
%sql duckdb:///:memory:
%sql conn --alias duckdb

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

Connecting to 'duckdb:///:memory:'

In [2]:
%%sql
CREATE TABLE CollectedResults (
   RACE                 TEXT
  ,YEAR                 INTEGER
  ,STAGE_ID             TEXT
  ,STAGE_DATE           DATE
  ,STAGE_NAME           TEXT
  ,STAGE_PROFILE        TEXT
  ,STAGE_TYPE           TEXT
  ,RANK                 TEXT
  ,RIDER                TEXT
  ,TEAM                 TEXT
  ,UCI_POINTS           DOUBLE
  ,PCS_POINTS           DOUBLE
  ,UPDATE_TIMESTAMP     DATETIME
);

INSERT INTO CollectedResults
SELECT 
   RaceName
  ,RaceYear
  ,StageNumber
  ,CASE 
    WHEN LEN(StageDate) = 10 THEN CAST(StageDate AS DATE)
    ELSE MAKE_DATE(
      RaceYear, 
      CAST(SUBSTRING(StageDate, 4, 2) AS INTEGER), 
      CAST(SUBSTRING(StageDate, 1, 2) AS INTEGER)
    )
   END
  ,StageName
  ,''
  ,StageType
  ,RiderRank
  ,TRIM(SUBSTRING(RiderName, 1, POSITION(TRIM(TeamName) IN RiderName) - 1))
  ,TeamName
  ,TRY_CAST(UCIPoints AS DOUBLE)
  ,TRY_CAST(PCSPoints AS DOUBLE)
  ,UpdateTimeStamp
FROM read_csv_auto("../data/results_races/*.csv", union_by_name = True);

UPDATE CollectedResults src
SET STAGE_DATE = (
  SELECT MAX(sub.STAGE_DATE)
  FROM CollectedResults sub
  WHERE sub.RACE = src.RACE AND sub.YEAR = src.YEAR
)
WHERE src.STAGE_NAME = 'GC' AND src.STAGE_DATE IS NULL;

COPY CollectedResults TO '../data/results/results_2025_full.csv' (HEADER, DELIMITER ',');

,Count
0,31866


In [3]:
%%sql
CREATE TABLE Riders             AS FROM "../data/riders.csv";
CREATE TABLE RidersSelectedBy   AS FROM "../data/riders_selected_by.csv";
CREATE TABLE RiderTeams         AS FROM "../data/rider_teams.csv";
CREATE TABLE Managers           AS FROM "../data/managers.csv";
CREATE TABLE ManagerTeams       AS FROM "../data/manager_teams.csv";
CREATE TABLE ManagerTeamsCheapo AS FROM "../data/manager_cheapo_teams.csv";
CREATE TABLE PointsSystem       AS FROM "../data/points_system.csv";
CREATE TABLE Races              AS FROM "../data/races.csv";
CREATE TABLE Results_2023       AS FROM "../data/results/results_2023_full.csv";
CREATE TABLE Results_2024       AS FROM "../data/results/results_2024_full.csv";
CREATE TABLE Results_2025       AS FROM "../data/results/results_2025_full.csv";
CREATE TABLE CheapoBans         AS FROM "../data/cheapo_bans.csv";

,Count
0,14


In [4]:
%%sql
CREATE OR REPLACE TABLE RaceResultsPoints AS
SELECT
     results.Race
    ,races.RaceName
    ,races.RaceCategory
    ,results.Stage_ID
    ,COALESCE(results.Stage_Date, races.RaceStart) AS Stage_Date
    ,results.Stage_Type
    ,CASE WHEN results.Stage_Name <> 'GC' AND (results.Stage_Type <> 'TTT' OR results.Stage_Type IS NULL) THEN 1 ELSE 0 END AS Løbsdage
    ,results.Rider AS RiderName_PCS
    ,riders.RiderName_Zweeler
    ,riders.RiderPrice
    ,riders.RiderTeam
    ,COALESCE(selected.ValgtAf, 0) AS SelectedBy
    ,CASE WHEN cbans.RiderName IS NULL AND riders.RiderPrice <= 2.5 THEN 'Ja' ELSE '' END AS Cheapo
    ,results.Rank
    ,points.RacePoints AS Points
    ,CASE WHEN races.RaceName = 'Tour Down Under' THEN NULL Else points.RacePoints END AS CheapoPoints
FROM Riders riders
LEFT JOIN Results_2025 results ON results.Rider = riders.RiderName_PCS
LEFT JOIN Races races ON races.RaceName_PCS = results.Race
LEFT JOIN CheapoBans cbans ON cbans.RiderName = riders.RiderName_Zweeler
LEFT JOIN RidersSelectedBy selected ON selected.RiderName_Zweeler = riders.RiderName_Zweeler
LEFT JOIN PointsSystem points ON 1 = 1
    AND (results.Stage_Type IS NULL OR results.Stage_Type <> 'TTT')
    AND points.RaceRank = TRY_CAST(results.Rank AS INTEGER) 
    AND CASE WHEN results.Stage_Name LIKE '%stage%' OR results.Stage_Name LIKE '%prologue%' THEN REPLACE(races.RaceCategory, 'Tour', 'Stage') ELSE races.RaceCategory END = points.RaceCategory;

,Count
0,28321


In [5]:
%%sql
rider_output << 
WITH
ManagerLists AS
(
    SELECT RiderName, STRING_AGG(ManagerName, ', ' ORDER BY ManagerName ASC) AS Managers
    FROM ManagerTeams
    GROUP BY RiderName
),
RiderOverview AS 
(
    SELECT
         RiderName_Zweeler AS Navn
        ,MAX(RiderTeam) AS Hold
        ,MAX(RiderPrice) AS Pris
        ,MAX(SelectedBy) AS Valgt
        ,CAST(SUM(Løbsdage) AS int) AS Løbsdage
        ,COALESCE(CAST(SUM(Points) AS int), 0) AS Point
    FROM RaceResultsPoints
    GROUP BY RiderName_Zweeler
)
SELECT 
     *
    ,ROUND(Point / Pris, 1) AS "P/mil"
    ,ROUND(CASE WHEN Løbsdage = 0 THEN 0 ELSE Point / Løbsdage END, 1) AS "P/dage"
    ,COALESCE((SELECT Managers FROM ManagerLists sub WHERE sub.RiderName = src.Navn), '') AS Managers
FROM RiderOverview src;

In [6]:
%%sql
rider_output_cheapo << 
WITH
ManagerLists AS
(
    SELECT RiderName, STRING_AGG(ManagerName, ', ' ORDER BY ManagerName ASC) AS Managers
    FROM ManagerTeamsCheapo
    GROUP BY RiderName
),
RiderOverview AS 
(
    SELECT
         RiderName_Zweeler AS Navn
        ,MAX(RiderTeam) AS Hold
        ,MAX(RiderPrice) AS Pris
        ,MAX(SelectedBy) AS Valgt
        ,CAST(SUM(Løbsdage) AS int) AS Løbsdage
        ,COALESCE(CAST(SUM(CheapoPoints) AS int), 0) AS Point
        ,MAX(Cheapo) AS Cheapo
    FROM RaceResultsPoints
    GROUP BY RiderName_Zweeler
)
SELECT 
     *
    ,ROUND(Point / Pris, 1) AS "P/mil"
    ,ROUND(CASE WHEN Løbsdage = 0 THEN 0 ELSE Point / Løbsdage END, 1) AS "P/dage"
    ,COALESCE((SELECT Managers FROM ManagerLists sub WHERE sub.RiderName = src.Navn), '') AS Managers
FROM RiderOverview src
WHERE Cheapo = 'Ja';

In [7]:
%%sql
WITH Src AS
(
    SELECT YEARWEEK(results.Stage_Date) AS Uge, teams.ManagerName AS Manager, CAST(SUM(Points) AS int) AS Point
    FROM RaceResultsPoints results
    INNER JOIN ManagerTeams teams ON teams.RiderName = results.RiderName_Zweeler
    WHERE 1 = 1
        AND Uge IS NOT NULL
    GROUP BY YEARWEEK(results.Stage_Date), teams.ManagerName
)
PIVOT Src
ON Uge
USING COALESCE(SUM(Point)::int, 0)
GROUP BY Manager
ORDER BY Manager;

,Manager,202504,202505,202506,202507,202508,202509,202510,202511,202512,...,202524,202525,202527,202528,202529,202530,202531,202532,202533,202534
0,Chrelle,959.0,170.0,144.0,210.0,1701.0,60.0,327.0,1480.0,125.0,...,1129.0,1443.0,433.0,1701.0,1527.0,1928.0,52.0,504.0,118.0,1149.0
1,Hustlersen,1420.0,26.0,35.0,337.0,1114.0,0.0,178.0,749.0,127.0,...,895.0,684.0,416.0,1638.0,1836.0,1899.0,100.0,326.0,86.0,968.0
2,Jappo,447.0,276.0,54.0,210.0,1779.0,213.0,501.0,1376.0,141.0,...,1275.0,335.0,492.0,1862.0,1480.0,1589.0,51.0,316.0,171.0,1342.0
3,Jarma,44.0,105.0,92.0,72.0,1391.0,169.0,172.0,890.0,85.0,...,1197.0,393.0,648.0,1837.0,1319.0,1660.0,41.0,544.0,178.0,1075.0
4,Kenk,899.0,49.0,272.0,206.0,1450.0,104.0,256.0,1445.0,167.0,...,1051.0,831.0,323.0,1761.0,1121.0,1564.0,203.0,945.0,58.0,1101.0
5,Knak,433.0,292.0,62.0,74.0,1494.0,96.0,449.0,998.0,153.0,...,1272.0,552.0,856.0,2002.0,1675.0,2115.0,17.0,122.0,70.0,1011.0
6,Matti,227.0,112.0,540.0,224.0,1804.0,57.0,133.0,1531.0,118.0,...,1204.0,675.0,333.0,1951.0,1208.0,1363.0,85.0,250.0,61.0,1338.0
7,Okholm,506.0,50.0,166.0,22.0,1604.0,171.0,176.0,1449.0,101.0,...,1619.0,549.0,532.0,2456.0,2241.0,2478.0,18.0,193.0,95.0,1055.0
8,Tommy,807.0,170.0,88.0,189.0,1158.0,126.0,363.0,1263.0,160.0,...,1045.0,347.0,354.0,1545.0,1520.0,1430.0,0.0,560.0,58.0,964.0
9,Visti,329.0,106.0,238.0,269.0,1312.0,104.0,262.0,1537.0,192.0,...,1203.0,657.0,458.0,2070.0,1786.0,2023.0,20.0,406.0,166.0,1494.0


In [8]:
%%sql
dagens_rapport_avanceret <<
WITH Src AS
(
     SELECT 
           CAST(results.Stage_Date AS date) AS Dato
          ,managers.ManagerName AS Manager
          ,CAST(SUM(results.Løbsdage) AS int) AS Løbsdage
          ,SUM(CASE WHEN results.Løbsdage = 1 THEN results.RiderPrice END) AS Millioner
          ,CAST(SUM(results.Points) AS int) AS Point
     FROM RaceResultsPoints results
     LEFT JOIN ManagerTeams managers ON managers.RiderName = results.RiderName_Zweeler
     GROUP BY
           CAST(results.Stage_Date AS DATE)
          ,managers.ManagerName
), ManagersDatoer AS
(
     SELECT 
           Manager
          ,Dato FROM
     (SELECT DISTINCT Manager FROM Src) m
     CROSS JOIN (SELECT DISTINCT Dato FROM Src) d
     WHERE m.Manager IS NOT NULL AND d.Dato IS NOT NULL
)
SELECT
      managersdatoer.Manager
     ,managersdatoer.Dato
     ,COALESCE(summer.Løbsdage, 0) AS Løbsdage
     ,COALESCE(summer.Millioner, 0) AS Millioner
     ,COALESCE(summer.Point, 0) AS Point
     ,ROUND(CASE WHEN COALESCE(summer.Løbsdage, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Løbsdage, 0) END, 2) AS "P/rytter"
     ,ROUND(CASE WHEN COALESCE(summer.Millioner, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Millioner, 0) END, 2) AS "P/mil"
     ,(
          SELECT COALESCE(STRING_AGG(RiderPoints, ', ' ORDER BY Points DESC), '')
          FROM
               (   SELECT
                         CONCAT(
                              SUBSTRING(r.RiderName_Zweeler, 1, position(', ' IN r.RiderName_Zweeler) - 1), 
                              ' (',
                              CAST(SUM(r.Points) AS int), 
                              ')'
                         ) AS RiderPoints,
                         SUM(r.Points) AS Points
                    FROM RaceResultsPoints r
                    INNER JOIN ManagerTeams m ON m.RiderName = r.RiderName_Zweeler
                    WHERE 1 = 1
                         AND CAST(r.Stage_Date AS date) = managersdatoer.Dato
                         AND m.ManagerName = managersdatoer.Manager
                         AND r.Points IS NOT NULL
                    GROUP BY r.RiderName_Zweeler
                    ORDER BY SUM(r.Points) DESC
                    LIMIT 3
               ) t
      ) AS "Top 3 ryttere"
FROM ManagersDatoer managersdatoer
LEFT JOIN Src summer ON managersdatoer.Manager = summer.Manager AND managersdatoer.Dato = summer.Dato
ORDER BY 
      managersdatoer.Dato DESC
     ,COALESCE(summer.Point, 0) DESC;

In [9]:
%%sql
ugens_rapport_avanceret <<
WITH Src AS
(
     SELECT 
           YEARWEEK(CAST(results.Stage_Date AS date)) AS Uge
          ,managers.ManagerName AS Manager
          ,CAST(SUM(results.Løbsdage) AS int) AS Løbsdage
          ,SUM(CASE WHEN results.Løbsdage = 1 THEN results.RiderPrice END) AS Millioner
          ,CAST(SUM(results.Points) AS int) AS Point
     FROM RaceResultsPoints results
     LEFT JOIN ManagerTeams managers ON managers.RiderName = results.RiderName_Zweeler
     GROUP BY
           YEARWEEK(CAST(results.Stage_Date AS date))
          ,managers.ManagerName
), ManagersDatoer AS
(
     SELECT 
           Manager
          ,Uge FROM
     (SELECT DISTINCT Manager FROM Src) m
     CROSS JOIN (SELECT DISTINCT Uge FROM Src) d
     WHERE m.Manager IS NOT NULL AND d.Uge IS NOT NULL
)
SELECT
      managersdatoer.Manager
     ,managersdatoer.Uge
     ,COALESCE(summer.Løbsdage, 0) AS Løbsdage
     ,COALESCE(summer.Millioner, 0) AS Millioner
     ,COALESCE(summer.Point, 0) AS Point
     ,ROUND(CASE WHEN COALESCE(summer.Løbsdage, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Løbsdage, 0) END, 2) AS "P/rytter"
     ,ROUND(CASE WHEN COALESCE(summer.Millioner, 0) = 0 THEN 0 ELSE COALESCE(summer.Point, 0) / COALESCE(summer.Millioner, 0) END, 2) AS "P/mil"
     ,(
          SELECT COALESCE(STRING_AGG(RiderPoints, ', ' ORDER BY Points DESC), '')
          FROM
               (   SELECT
                         CONCAT(
                              SUBSTRING(r.RiderName_Zweeler, 1, position(', ' IN r.RiderName_Zweeler) - 1), 
                              ' (',
                              CAST(SUM(r.Points) AS int), 
                              ')'
                         ) AS RiderPoints,
                         SUM(r.Points) AS Points
                    FROM RaceResultsPoints r
                    INNER JOIN ManagerTeams m ON m.RiderName = r.RiderName_Zweeler
                    WHERE 1 = 1
                         AND YEARWEEK(CAST(r.Stage_Date AS date)) = managersdatoer.Uge
                         AND m.ManagerName = managersdatoer.Manager
                         AND r.Points IS NOT NULL
                    GROUP BY r.RiderName_Zweeler
                    ORDER BY SUM(r.Points) DESC
                    LIMIT 3
               ) t
      ) AS "Top 3 ryttere"
FROM ManagersDatoer managersdatoer
LEFT JOIN Src summer ON managersdatoer.Manager = summer.Manager AND managersdatoer.Uge = summer.Uge
ORDER BY 
      managersdatoer.Uge DESC
     ,COALESCE(summer.Point, 0) DESC;

In [10]:
rider_output.to_html("../outputtables/RytteroversigtTabel.html", table_id = "filterabletable", index = False)
rider_output_cheapo.to_html("../outputtables/RytteroversigtCheapoTabel.html", table_id = "filterabletable", index = False)
dagens_rapport_avanceret.to_html("../outputtables/DagensRapportAvanceret.html", table_id = "filterabletable", index = False)
ugens_rapport_avanceret.to_html("../outputtables/UgensRapportAvanceret.html", table_id = "filterabletable", index = False)

In [11]:
%%sql
WITH Src AS
(
    SELECT 
        managers.ManagerName AS Manager
        ,YEAR(results.Stage_Date) * 100 + MONTH(results.Stage_Date) AS ÅrMåned
        ,SUM(results.Løbsdage * results.RiderPrice) AS SamletPris
        ,SUM(results.Løbsdage) AS SamledeLøbsdage
        ,SUM(results.Points) AS SamledePoint
    FROM RaceResultsPoints results
    LEFT JOIN ManagerTeams managers ON managers.RiderName = results.RiderName_Zweeler
    WHERE 1 = 1
        AND managers.ManagerName IS NOT NULL
    GROUP BY
        managers.ManagerName
        ,YEAR(results.Stage_Date) * 100 + MONTH(results.Stage_Date)
), Managers AS
(
    SELECT DISTINCT Manager
    FROM Src
)
SELECT 
     m.Manager
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202501), 0), 2) AS "202501"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202502), 0), 2) AS "202502"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202503), 0), 2) AS "202503"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202504), 0), 2) AS "202504"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202505), 0), 2) AS "202505"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202506), 0), 2) AS "202506"
    ,ROUND(COALESCE((SELECT SamledePoint / SamletPris FROM Src s WHERE s.Manager = m.Manager AND s.ÅrMåned = 202507), 0), 2) AS "202507"
    ,ROUND(COALESCE((SELECT SUM(SamletPris) FROM Src s WHERE s.Manager = m.Manager), 0), 2) AS "Millioner i alt"
    ,CAST(ROUND(COALESCE((SELECT SUM(SamledePoint) FROM Src s WHERE s.Manager = m.Manager), 0), 2) AS int) AS "Point i alt"
    ,ROUND(COALESCE((SELECT SUM(SamledePoint) / SUM(SamletPris) FROM Src s WHERE s.Manager = m.Manager), 0), 2) AS "Total"
FROM Managers m;

,Manager,202501,202502,202503,202504,202505,202506,202507,Millioner i alt,Point i alt,Total
0,Visti,2.57,1.06,1.34,1.38,1.89,1.28,1.33,14576.0,20344,1.40
1,Chrelle,4.04,1.34,1.39,1.76,1.79,1.71,1.41,13362.1,21675,1.62
2,Tommy,2.92,1.16,1.32,1.37,1.78,1.42,1.13,12747.1,17527,1.37
3,Knak,2.04,1.15,1.18,1.52,1.68,1.24,1.22,14852.3,19731,1.33
4,Hustlersen,4.12,1.18,1.27,1.34,1.21,1.23,1.25,13687.8,18518,1.35
5,Jarma,0.37,1.15,1.09,1.65,1.76,0.98,1.37,11435.2,14973,1.31
6,Okholm,2.70,1.19,1.36,1.15,1.86,1.39,1.56,13432.5,19704,1.47
7,Kenk,4.80,1.15,1.39,1.37,1.52,1.30,1.23,14315.0,19896,1.39
8,Matti,2.95,1.48,1.28,1.17,1.85,1.38,1.12,13561.5,18091,1.33
9,Jappo,2.50,1.33,1.37,1.24,1.75,1.21,1.13,14842.6,20077,1.35
